# DataFrame Operations

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

In [2]:
## To display notebook cell with horizontal scroll bar
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

#### Start a simple Spark Session

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType

spark = SparkSession.builder \
    .master("local") \
    .getOrCreate()

In [17]:
spark

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [11]:
df = spark.read.csv("work/walmart_stock.csv", header=True, 
                    schema="""Date STRING, Open FLOAT, High FLOAT, Low FLOAT, Close FLOAT, Volume INT, Adj_Close FLOAT""")


#### What are the column names?

In [12]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj_Close']

#### What does the Schema look like?

In [13]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj_Close: float (nullable = true)



#### Print out the first 5 salary.

In [14]:
df.show(5)

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj_Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|
+----------+-----+-----+-----+-----+--------+---------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [15]:
df.describe().show()

+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|             Open|             High|              Low|            Close|           Volume|        Adj_Close|
+-------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      NULL|72.35785375452572| 72.8393880756178|71.91860094964979|72.38844997363553|8222093.481717011|67.23883840200064|
| stddev|      NULL|6.768090251767697|6.768186825250206|6.744075739203606|6.756859160119612|  4519780.8431556|6.722609385249684|
|    min|2012-01-03|            56.39|            57.06|             56.3|            56.42|          2094900|         50.36369|
|    max|2016-12-30|             90.8|            90.97|            89.25|            90.47|     

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. 

If you get stuck on this, don't worry, just view the solutions.

In [18]:
df_describe = df.describe()

numeric_cols = [col_name for col_name in df_describe.columns if col_name not in ['summary','Date']]


for col_name in numeric_cols:
    df_describe = df_describe.withColumn(col_name, round(col(col_name), 2))

In [19]:
df_describe.show()

+-------+----------+------+------+------+------+----------+---------+
|summary|      Date|  Open|  High|   Low| Close|    Volume|Adj_Close|
+-------+----------+------+------+------+------+----------+---------+
|  count|      1258|1258.0|1258.0|1258.0|1258.0|    1258.0|   1258.0|
|   mean|      NULL| 72.36| 72.84| 71.92| 72.39|8222093.48|    67.24|
| stddev|      NULL|  6.77|  6.77|  6.74|  6.76|4519780.84|     6.72|
|    min|2012-01-03| 56.39| 57.06|  56.3| 56.42| 2094900.0|    50.36|
|    max|2016-12-30|  90.8| 90.97| 89.25| 90.47| 8.08981E7|    84.91|
+-------+----------+------+------+------+------+----------+---------+



#### ################################################################## End of Bonus Question

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [20]:
new_df = df.withColumn("HV_Ratio", df["High"] / df["Volume"])
new_df.show(5)

+----------+-----+-----+-----+-----+--------+---------+--------------------+
|      Date| Open| High|  Low|Close|  Volume|Adj_Close|            HV_Ratio|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|4.819714682786927E-6|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|6.290848662516662E-6|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554| 4.66941298944916E-6|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922| 7.36733843444859E-6|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|8.915604814435727E-6|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [ ]:
max_high = new_df.select(max("High").alias("Peak_High")).first()[0]
max_high


90.97000122070312

In [22]:
new_df.select("Date" , "High").where(new_df["High"] == max_high).show()

+----------+-----+
|      Date| High|
+----------+-----+
|2015-01-13|90.97|
+----------+-----+



#### What is the mean of the Close column?

In [23]:
new_df.select(format_number(mean("Close"),2).alias("Close_Mean")).show()

+----------+
|Close_Mean|
+----------+
|     72.39|
+----------+



#### What is the max and min of the Volume column?

In [24]:
new_df.select(max("Volume").alias("Max_Volume"), min("Volume").alias("Min_Volume")).show()

+----------+----------+
|Max_Volume|Min_Volume|
+----------+----------+
|  80898100|   2094900|
+----------+----------+



#### How many days was the Close lower than 60 dollars?

In [25]:
new_df.filter(new_df["High"] < 60).count()

64

In [26]:
new_df.select("Date").filter(new_df["High"] < 60).count()

64

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [27]:
new_df.filter(new_df["High"] > 80).count() / new_df.count()

0.09141494435612083

#### What is the Pearson correlation between High and Volume?

In [28]:
new_df.stat.corr("High" , "Volume")

-0.3384326061799556

#### What is the max High per year?

In [29]:
new_df.select(year("Date").alias("Year"),"High") \
      .groupBy("Year") \
      .agg(max("High").alias("Max_High")) \
      .orderBy("Year").show()

+----+--------+
|Year|Max_High|
+----+--------+
|2012|    77.6|
|2013|   81.37|
|2014|   88.09|
|2015|   90.97|
|2016|   75.19|
+----+--------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [30]:
new_df.select(month("Date").alias("Month") , "Close") \
        .groupBy("Month") \
        .agg(avg("Close").alias("Avg_Close")) \
        .orderBy("Month").show()

+-----+-----------------+
|Month|        Avg_Close|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+



## Task 2


### Read "test1" dataset:

In [34]:
test1 = spark.read.csv("work/test1.csv", header=True)
test1.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



### Display Salary of the people less than or equal to 20000

In [35]:
test1.select("Name", "Salary").where(test1["Salary"] <= 20000).show()

+-------+------+
|   Name|Salary|
+-------+------+
|  Sunny| 20000|
|   Paul| 20000|
| Harsha| 15000|
|Shubham| 18000|
+-------+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [36]:
test1.select("Name", "Salary").where((test1["Salary"] <= 20000) & (test1["Salary"] >= 15000)).show()

+-------+------+
|   Name|Salary|
+-------+------+
|  Sunny| 20000|
|   Paul| 20000|
| Harsha| 15000|
|Shubham| 18000|
+-------+------+



## Task 3 

### Read "test3" dataset:

In [37]:
test3 = spark.read.csv("work/test3.csv", header = True)


### Display dataset

In [38]:
test3.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



### Display schema

In [39]:
test3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: string (nullable = true)



### Group by "Name" column and using sum function on "salary" column

In [40]:
test3.groupBy("Name").agg(sum("Salary").alias("Total_Salary")).show()

+---------+------------+
|     Name|Total_Salary|
+---------+------------+
|Sudhanshu|     35000.0|
|    Sunny|     12000.0|
|    Krish|     19000.0|
|   Mahesh|      7000.0|
+---------+------------+



### Group by "Name" column and using avg function on "salary" column

In [41]:
test3.groupBy("Name").agg(avg("Salary").alias("Average_Salary")).show()

+---------+------------------+
|     Name|    Average_Salary|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "salary" column

In [42]:
test3.groupBy("Departments").agg(sum("Salary").alias("Total_Salary")).show()

+------------+------------+
| Departments|Total_Salary|
+------------+------------+
|         IOT|     15000.0|
|    Big Data|     15000.0|
|Data Science|     43000.0|
+------------+------------+



### Group by "Departments" column and using mean function on "salary" column:

In [43]:
test3.groupBy("Name").agg(format_number(mean("Salary"),2).alias("Mean_Salary")).show()

+---------+-----------+
|     Name|Mean_Salary|
+---------+-----------+
|Sudhanshu|  11,666.67|
|    Sunny|   6,000.00|
|    Krish|   6,333.33|
|   Mahesh|   3,500.00|
+---------+-----------+



### Apply agg to using sum function get the total of salaries

In [44]:
test3.agg(sum("Salary").alias("Total_Salaries")).show()

+--------------+
|Total_Salaries|
+--------------+
|       73000.0|
+--------------+

